<a href="https://colab.research.google.com/github/JanneBckr/lense_cbg_meep_simulation/blob/main/Kopie_von_meep_lens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Colab Notebook to run FDTD simulations with meep

Author: Steffen Michaelis de Vasconcellos

## 0. Installation
First, we install CondaLab via pip to have the convenient Conda environment. It takes about 30-60 seconds. The session will send a message that it has crashed - this is normal. The environment is deleted each time you reconnect to Google Colab, so keep the runtime open.


In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  try:
    import condacolab
    condacolab.check()
  except ImportError:
    !pip install -q condacolab
    import condacolab
    condacolab.install()


✨🍰✨ Everything looks OK!


Then we install meep via anaconda. We don't want to see the output therefore we use %%capture. Takes about 1 or 2 minutes.

In [ ]:
%%capture
try:
  import meep as matplotlib
except ImportError:
  !conda install -c conda-forge pymeep

# Meep simulation

Now we can run a meep simulation. The meep documentation is found here: https://meep.readthedocs.io/en/latest/Python_User_Interface/

## 1. Import packages



In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import meep as mp

## 2. Define constants
Define some physical constants of the simulation (e.g. refractive index, wavelength) and some parameters of the FDTD simulation (e.g. resolution, size of the simulation grid)

Most important: Defines the parameters of the elliptical lens and the polarization of the dipole source. Here you can change things!

In [ ]:
#refractive indieces
index_lens = 1.55
index_substrate = 1.476
index_air = 1
index_hBN = 2.2
index_cbg = 1.51


#define lens parameters
lens_a = 7.5
lens_b = lens_a*index_lens/(index_lens**2-index_air**2)**(1/2)
lens_h = lens_a*((index_lens+index_air)/(index_lens-index_air))**(1/2)

lens_offset = 0
lens_offset_x = 0

print('Lens parameters:')
print('Lens short axis (a):', lens_a)
print('Lens long axis (b):', lens_b)
print('Lens height (h):', lens_h)
print('Lens offset (y):', lens_offset)
print('Lens offset (x):', lens_offset_x)


#define CBG parameters
cbg_radius = 4.8  # Define the radius of the CBG
num_rings = 10  # Define number of rings
ring_width = 0.24  # Width of each ring
air_gap = 0.160 #width of trenches between rings


#source polarization "s" or "p" or "z" or "45°"
source_direction = "s"

#wavelength in um
wavelength = 0.625


## Calculate and define some further constants for the simulation - don't edit unless you know what you are doing

#frequency
fcen = 1/wavelength

#resolution
resolution = 16

hBN_height = 2/resolution #center of the dipole emitter, 1 grid point above the substrate surface

#resolution is set via parameters alternative here:
#resolution_per_wavelength = 8
#resolution = int(np.floor(resolution_per_wavelength*fcen*index_lens))  # pixels/um
print("Resolution", resolution)

dpml = 0.5

#defince simulation cell size. size_x and y are parameters multiplies with lens size
size_y = 15
size_x = 8
sx = np.floor(size_x*(lens_a))+1  #size of simulation area
sy = np.floor(size_y*(lens_a))+1

substrate_height = sy/2 + dpml #- extra_space
substrate_x = sx + 2*dpml #- extra_space


Lens parameters:
Lens short axis (a): 7.5
Lens long axis (b): 9.816157693907416
Lens height (h): 16.14916265771865
Lens offset (y): 0
Lens offset (x): 0
Resolution 16


## 3. Define the geometry in meep
Here an ellipsoid is chosen for the lens. You could modify that part by using a different geomerty class (see meep documentation), but initally it is better to modify only parameters in the cell above.

In [ ]:
#define geometry
geometry = []

#select geometry:

#geom = 'lense'
geom = 'cbg'

if geom == 'lense':
    #lens
    geometry.append(
        mp.Ellipsoid(
        center=mp.Vector3(lens_offset_x,lens_b-lens_h-lens_offset,0),
        size=mp.Vector3(2*lens_a,2*lens_b,2*lens_b),
        e1=mp.Vector3(1,0,0), e2=mp.Vector3(0,1,0), e3=mp.Vector3(0,0,1),
        material=mp.Medium(index=index_lens)))

if geom == 'cbg':
    #Circular Bragg Cavity
    current_radius = cbg_radius

    # Assuming surrounding medium is air, hence material inside the cutout cylinder is air
    for i in range(num_rings):
        outer_radius = current_radius
        inner_radius = current_radius - ring_width
        material = mp.Medium(index=3.5) if i % 2 == 0 else mp.Medium(index=1.5)

        # Add outer cylinder
        geometry.append(mp.Cylinder(radius=outer_radius,
                                    height=mp.inf,
                                    material=index_cbg,
                                    center=mp.Vector3()))

        if i < num_rings - 1:  # No need for inner cutout for innermost ring
            # Add inner cylinder to create the ring
            geometry.append(mp.Cylinder(radius=inner_radius,
                                        height=mp.inf,
                                        material=mp.air,
                                        center=mp.Vector3()))

        current_radius -= (ring_width + air_gap)

#substrate
geometry.append(
    mp.Block(
        size=mp.Vector3(substrate_x, substrate_height, 0),
        center=mp.Vector3(0,substrate_height/2,0),
        material=mp.Medium(index=index_substrate)))


## 4. Define the dipole source in meep
Don't edit here.

In [ ]:
if source_direction == "p":
    print("Source direction Ex")
    src_cmpt = mp.Ex
elif source_direction == "s":
    print("Source direction Ez")
    src_cmpt = mp.Ez
elif source_direction == "z":
    print("Source direction perpendicular to surface (Ey)")
    src_cmpt = mp.Ey

sources = [mp.Source(src=mp.ContinuousSource(fcen),
                     center=mp.Vector3(0,-hBN_height/2,0),
                     component=src_cmpt)]

Source direction Ez


## 5. Setup the simulation
Don't edit here. Running this cell resets also the simulation.

In [ ]:
#calculate cell
cell = mp.Vector3(sx+2*dpml,sy+2*dpml)  #size of computation cell

#make PML
pml_layers = [mp.PML(dpml)]

#add symmetries
#in a 2D simulation (optical axis in Y direction), we have only symmetires in X direction. This can be either inverted (phase=-1) or normal.
if src_cmpt == mp.Ex:
    symmetries = [mp.Mirror(mp.X,phase=-1)]
elif src_cmpt == mp.Ey:
    symmetries = [mp.Mirror(mp.X,phase=+1)]
elif src_cmpt == mp.Ez:
    symmetries = [mp.Mirror(mp.X,phase=+1)]

if lens_offset_x != 0:
  symmetries = []

#make simulation
sim = mp.Simulation(cell_size=cell,
                    resolution=resolution,
                    sources=sources,
                    geometry=geometry,
                    symmetries=symmetries,
                    boundary_layers=pml_layers,
                    force_complex_fields=False)



## 6. Run the simulation
Running for a certain amount of timesteps (e.g. 100). You can run this cell multiple times to continoue the simulation. `until` means that that number of step is added to the simulation. The number of timesteps required that the light propagates through the whole cell depends on the size of the cell and the resolution

In [ ]:
run_until = 10
sim.run(until=run_until)

## 7. Analyze

After running the simulation, we can analyze the output. Here we plot the total power of the electric field overlayed with epsilon.

Edit vmax if the color scale is not appropriate.

You can also plot other parameters, e.g. the electric field. Try uncommenting the lines.

In [ ]:
eps_data = sim.get_array(center=mp.Vector3(), size=sim.cell_size, component=mp.Dielectric).transpose()
#efield_data = sim.get_array(center=mp.Vector3(), size=sim.cell_size, component=src_cmpt).transpose()
tot_pwr = sim.get_tot_pwr().transpose()

plt.figure(figsize=(10,8))
plt.imshow(eps_data, interpolation='spline36', cmap='binary')
plt.imshow(tot_pwr, interpolation='spline36', cmap='hot', vmin=0,vmax=.5,alpha=0.8)
#plt.imshow(efield_data, interpolation='spline36', cmap='RdBu', vmin=-.2,vmax=.2,alpha=0.8)
plt.axis('off')
plt.show()

NameError: name 'sim' is not defined